<a href="https://colab.research.google.com/github/Aniket99coder/EplaneEnergyOpt/blob/main/EnergyOptEplane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 59.9 MB/s eta 0:00:00


In [2]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_Time_Based", LpMinimize)

# Parameters (based on the image data)
battery_capacity = 21  # in kWh
soc_max_taxi = 3 / 100  # Convert percentage to fraction
soc_max_climb = 9 / 100
soc_max_descend = 3 / 100
soc_max_cruise = 48 / 100
roc_max_climb = 71  # in ft/min
altitude_cruise = 800  # in ft

# Time step in hours
delta_t = 1 / 60  # 1 minute in hours

# Total duration (in minutes) for each stage, assuming typical values within provided bounds
duration_taxi = 10  # Taxi for 10 minutes
duration_climb = 5  # Climb for 5 minutes
duration_descend = 5  # Descend for 5 minutes
duration_cruise = 30  # Cruise for 30 minutes

# Number of time intervals for each stage
n_taxi = int(duration_taxi / (delta_t * 60))
n_climb = int(duration_climb / (delta_t * 60))
n_descend = int(duration_descend / (delta_t * 60))
n_cruise = int(duration_cruise / (delta_t * 60))

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 0, 67
power_min_climb, power_max_climb = 0, 71
power_min_descend, power_max_descend = 0, 60
power_min_cruise, power_max_cruise = 0, 50

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(n_taxi)]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(n_climb)]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(n_descend)]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(n_cruise)]

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_descend]) + \
         lpSum([P * delta_t for P in P_cruise])

# Constraints

# Battery capacity constraint (total energy used must not exceed battery capacity)
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_descend]) + \
         lpSum([P * delta_t for P in P_cruise]) <= battery_capacity

# SoC drop constraints for each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity <= soc_max_taxi
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity <= soc_max_climb
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity <= soc_max_descend
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity <= soc_max_cruise

# Additional constraints for rate of climb (for climb stage)
# Assuming a relationship or limitation based on max RoC in ft/min; placeholder constraint
model += lpSum(P_climb) / n_climb <= roc_max_climb  # This is a placeholder; adjust based on actual data

# Altitude constraint for cruise
# Assuming a relationship where cruise power should sustain altitude; placeholder
model += lpSum(P_cruise) / n_cruise >= 0.5 * altitude_cruise  # Placeholder relationship; replace as needed

# Solve the problem
model.solve()

# Print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("\nTotal Energy Consumption:", value(model.objective), "kWh")


Status: -1
Optimal Power Settings per Time Step:
Taxi Power: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] kW
Climb Power: [0.0, 0.0, 0.0, 0.0, 0.0] kW
Descend Power: [0.0, 0.0, 0.0, 0.0, 0.0] kW
Cruise Power: [11400.0, 50.0, 0.0, 0.0, 0.0, 50.0, 0.0, 0.0, 50.0, 50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 50.0, 50.0, 50.0, 0.0, 50.0, 0.0, 0.0, 50.0, 50.0, 0.0, 50.0, 0.0, 50.0, 0.0, 0.0] kW

Total Energy Consumption: 200.0000000000001 kWh


## Simple Model (Few constraints )
- Objective: Minimize total energy consumption.
- Battery Capacity Constraint: Ensure the total energy does not exceed the battery capacity.
- Power Bounds for Each Stage: Ensure the power for each stage is within specified minimum and maximum bounds.


In [3]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 1, 10  # Set minimum power to ensure operation
power_min_climb, power_max_climb = 50, 70  # Set minimum power for climb phase
power_min_cruise, power_max_cruise = 10, 50  # Minimum cruise power to sustain flight
power_min_descend, power_max_descend = 5, 25  # Descent might require less power

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.277  # Min and max duration for taxi in hours
duration_climb_min, duration_climb_max = 0.006, 0.047
duration_cruise_min, duration_cruise_max = 0.003, 0.503
duration_descend_min, duration_descend_max = 0.005, 0.052

# SoC drop limits (percent of battery capacity) for each stage
soc_max_drop_taxi = 3  # Maximum 3% drop in Taxi
soc_max_drop_climb = 9  # Maximum 9% drop in Climb
soc_max_drop_cruise = 48  # Maximum 48% drop in Cruise
soc_max_drop_descend = 3  # Maximum 3% drop in Descend

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint (total energy used must not exceed battery capacity)
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints to avoid trivial zero power usage
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
# Calculate cumulative energy used in each stage and ensure it doesn't exceed max SoC drop

# Taxi stage SoC drop constraint
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
# Climb stage SoC drop constraint
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
# Cruise stage SoC drop constraint
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
# Descend stage SoC drop constraint
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"


# Solve the problem
model.solve()

# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")


Status: 1
Optimal Power Settings per Time Step:
Taxi Power: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] kW
Climb Power: [50.0, 50.0] kW
Cruise Power: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] kW
Descend Power: [5.0, 5.0, 5.0] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.26666667 hours ( 16.0 minutes)
Climb Duration: 0.033333333 hours ( 2.0 minutes)
Cruise Duration: 0.5 hours ( 30.0 minutes)
Descend Duration: 0.05 hours ( 3.0 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		1.00		99.92
Taxi	2		1.00		99.84
Taxi	3		1.00		99.76
Taxi	4		1.00		99.68
Taxi	5		1.00		99.60
Taxi	6		1.00		99.52
Taxi	7		1.00		99.44
Taxi	8		1.00		99.37
Taxi	9		1.00		99.29
Taxi	10		1.00		99.21
Taxi	11		1.00		99.13
Taxi	12		1.00		99.05
Taxi	13		1.00		98.97
Taxi	14		1.00		9

In [4]:
# Print specific SoC drop constraints
for name, constraint in model.constraints.items():
    if "SoC_Drop" in name:
        print(f"{name}: {constraint}")


SoC_Drop_Taxi: 0.07936507936507936*Power_Taxi_0 + 0.07936507936507936*Power_Taxi_1 + 0.07936507936507936*Power_Taxi_10 + 0.07936507936507936*Power_Taxi_11 + 0.07936507936507936*Power_Taxi_12 + 0.07936507936507936*Power_Taxi_13 + 0.07936507936507936*Power_Taxi_14 + 0.07936507936507936*Power_Taxi_15 + 0.07936507936507936*Power_Taxi_2 + 0.07936507936507936*Power_Taxi_3 + 0.07936507936507936*Power_Taxi_4 + 0.07936507936507936*Power_Taxi_5 + 0.07936507936507936*Power_Taxi_6 + 0.07936507936507936*Power_Taxi_7 + 0.07936507936507936*Power_Taxi_8 + 0.07936507936507936*Power_Taxi_9 <= 3.0
SoC_Drop_Climb: 0.07936507936507936*Power_Climb_0 + 0.07936507936507936*Power_Climb_1 <= 9.0
SoC_Drop_Cruise: 0.07936507936507936*Power_Cruise_0 + 0.07936507936507936*Power_Cruise_1 + 0.07936507936507936*Power_Cruise_10 + 0.07936507936507936*Power_Cruise_11 + 0.07936507936507936*Power_Cruise_12 + 0.07936507936507936*Power_Cruise_13 + 0.07936507936507936*Power_Cruise_14 + 0.07936507936507936*Power_Cruise_15 + 0.

In [5]:
# Print the values of all decision variables
print("\nValues of Decision Variables:")
for variable in model.variables():
    print(f"{variable.name} = {value(variable)}")


Values of Decision Variables:
Duration_Climb = 0.033333333
Duration_Cruise = 0.5
Duration_Descend = 0.05
Duration_Taxi = 0.26666667
Power_Climb_0 = 50.0
Power_Climb_1 = 50.0
Power_Cruise_0 = 10.0
Power_Cruise_1 = 10.0
Power_Cruise_10 = 10.0
Power_Cruise_11 = 10.0
Power_Cruise_12 = 10.0
Power_Cruise_13 = 10.0
Power_Cruise_14 = 10.0
Power_Cruise_15 = 10.0
Power_Cruise_16 = 10.0
Power_Cruise_17 = 10.0
Power_Cruise_18 = 10.0
Power_Cruise_19 = 10.0
Power_Cruise_2 = 10.0
Power_Cruise_20 = 10.0
Power_Cruise_21 = 10.0
Power_Cruise_22 = 10.0
Power_Cruise_23 = 10.0
Power_Cruise_24 = 10.0
Power_Cruise_25 = 10.0
Power_Cruise_26 = 10.0
Power_Cruise_27 = 10.0
Power_Cruise_28 = 10.0
Power_Cruise_29 = 10.0
Power_Cruise_3 = 10.0
Power_Cruise_4 = 10.0
Power_Cruise_5 = 10.0
Power_Cruise_6 = 10.0
Power_Cruise_7 = 10.0
Power_Cruise_8 = 10.0
Power_Cruise_9 = 10.0
Power_Descend_0 = 5.0
Power_Descend_1 = 5.0
Power_Descend_2 = 5.0
Power_Taxi_0 = 1.0
Power_Taxi_1 = 1.0
Power_Taxi_10 = 1.0
Power_Taxi_11 = 1.0
P

In [6]:
# Solve the model
model.solve()

# Check constraint satisfaction
print("\nConstraint Satisfaction Check:")
for name, constraint in model.constraints.items():
    lhs_value = constraint.value()
    if (constraint.sense == -1 and lhs_value > constraint.constant) or \
       (constraint.sense == 1 and lhs_value < constraint.constant) or \
       (constraint.sense == 0 and lhs_value != constraint.constant):
        print(f"Constraint '{name}' is not satisfied: LHS = {lhs_value}, RHS = {constraint.constant}")
    else:
        print(f"Constraint '{name}' is satisfied: LHS = {lhs_value}, RHS = {constraint.constant}")



Constraint Satisfaction Check:
Constraint 'Battery_Capacity' is not satisfied: LHS = -13.816666666666668, RHS = -21.0
Constraint 'Duration_Taxi' is not satisfied: LHS = -3.333333331578814e-09, RHS = 0.26666666666666666
Constraint 'Duration_Climb' is not satisfied: LHS = 3.333333331578814e-10, RHS = 0.03333333333333333
Constraint 'Duration_Cruise' is not satisfied: LHS = -5.551115123125783e-17, RHS = 0.49999999999999994
Constraint 'Duration_Descend' is not satisfied: LHS = 0.0, RHS = 0.05
Constraint 'Min_Power_Taxi_0' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_1' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_2' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_3' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_4' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_5' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_6' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_7' is satisfied: LHS = 0.0, RHS

## Model 2 ( Reach X altitude before cruising)

In [7]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_drop_rate = 2  # Minimum SoC drop rate in % per minute for climb

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 1, 10
power_min_climb, power_max_climb = 2, 70
power_min_cruise, power_max_cruise = 10, 50
power_min_descend, power_max_descend = 5, 25

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.277
duration_climb_min, duration_climb_max = 0.006, 0.047
duration_cruise_min, duration_cruise_max = 0.003, 0.503
duration_descend_min, duration_descend_max = 0.005, 0.052

# SoC drop limits for each stage
soc_max_drop_taxi = 3
soc_max_drop_climb = 9
soc_max_drop_cruise = 48
soc_max_drop_descend = 3

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"

# Altitude constraint: Ensure SoC drop during climb is enough to reach target altitude
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 * altitude_rate >= altitude_target, "Altitude_Climb_Target"

# Minimum SoC drop rate constraint for climb to achieve rate of altitude gain
for t in range(int(duration_climb_max / delta_t)):
    model += (P_climb[t] * delta_t / battery_capacity) * 100 >= min_soc_drop_rate, f"Min_SoC_Drop_Rate_Climb_{t}"

# Solve the problem
model.solve()

# Print results
# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")

Status: 1
Optimal Power Settings per Time Step:
Taxi Power: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] kW
Climb Power: [70.0, 39.2] kW
Cruise Power: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] kW
Descend Power: [5.0, 5.0, 5.0] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.26666667 hours ( 16.0 minutes)
Climb Duration: 0.033333333 hours ( 2.0 minutes)
Cruise Duration: 0.5 hours ( 30.0 minutes)
Descend Duration: 0.05 hours ( 3.0 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		1.00		99.92
Taxi	2		1.00		99.84
Taxi	3		1.00		99.76
Taxi	4		1.00		99.68
Taxi	5		1.00		99.60
Taxi	6		1.00		99.52
Taxi	7		1.00		99.44
Taxi	8		1.00		99.37
Taxi	9		1.00		99.29
Taxi	10		1.00		99.21
Taxi	11		1.00		99.13
Taxi	12		1.00		99.05
Taxi	13		1.00		98.97
Taxi	14		1.00		9

# 2. Using the whiskers from real data for power for `Model 2`

Power Statistics by Phase:

Phase: Cruise
- Median Power: 19.595 kW
- Average Power: 19.860 kW
- Minimum Power: 11.535 kW
- Maximum Power: 36.600 kW
- Q1 (25th Percentile): 17.293 kW
- Q3 (75th Percentile): 21.673 kW

----------------------------------------

Phase: Taxi
- Median Power: 1.150 kW
- Average Power: 1.218 kW
- Minimum Power: 0.000 kW
- Maximum Power: 8.954 kW
- Q1 (25th Percentile): 0.645 kW
- Q3 (75th Percentile): 1.477 kW

----------------------------------------

Phase: CLimb
- Median Power: 45.813 kW
- Average Power: 45.339 kW
- Minimum Power: 30.753 kW
- Maximum Power: 57.227 kW
- Q1 (25th Percentile): 42.715 kW
- Q3 (75th Percentile): 47.810 kW

----------------------------------------

Phase: Landing
- Median Power: 1.036 kW
- Average Power: 1.966 kW
- Minimum Power: 0.000 kW
- Maximum Power: 24.310 kW
- Q1 (25th Percentile): 0.338 kW
- Q3 (75th Percentile): 2.656 kW

----------------------------------------

In [8]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 200  # Feet gained per 1% SoC drop during climb
min_soc_drop_rate = 2  # Minimum SoC drop rate in % per minute for climb

# Time step in hours (1 minute in hours)
delta_t = 1 / 120

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 0.5, 2
power_min_climb, power_max_climb = 40, 60
power_min_cruise, power_max_cruise = 15, 22
power_min_descend, power_max_descend = 0.1, 3

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.122
duration_climb_min, duration_climb_max = 0.006, 0.033
duration_cruise_min, duration_cruise_max = 0.003, 0.029
duration_descend_min, duration_descend_max = 0.005, 0.04

# SoC drop limits for each stage
soc_max_drop_taxi = 2
soc_max_drop_climb = 8
soc_max_drop_cruise = 3
soc_max_drop_descend = 1

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"

# Altitude constraint: Ensure SoC drop during climb is enough to reach target altitude
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 * altitude_rate >= altitude_target, "Altitude_Climb_Target"

# Minimum SoC drop rate constraint for climb to achieve rate of altitude gain
for t in range(int(duration_climb_max / delta_t)):
    model += (P_climb[t] * delta_t / battery_capacity) * 100 >= min_soc_drop_rate, f"Min_SoC_Drop_Rate_Climb_{t}"

# Solve the problem
model.solve()

# Print results
# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")

Status: 1
Optimal Power Settings per Time Step:
Taxi Power: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5] kW
Climb Power: [60.0, 53.4, 50.4] kW
Cruise Power: [15.0, 15.0, 15.0] kW
Descend Power: [0.1, 0.1, 0.1, 0.1] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.11666667 hours ( 7.0 minutes)
Climb Duration: 0.025 hours ( 1.5 minutes)
Cruise Duration: 0.025 hours ( 1.5 minutes)
Descend Duration: 0.033333333 hours ( 2.0 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		0.50		99.98
Taxi	2		0.50		99.96
Taxi	3		0.50		99.94
Taxi	4		0.50		99.92
Taxi	5		0.50		99.90
Taxi	6		0.50		99.88
Taxi	7		0.50		99.86
Taxi	8		0.50		99.84
Taxi	9		0.50		99.82
Taxi	10		0.50		99.80
Taxi	11		0.50		99.78
Taxi	12		0.50		99.76
Taxi	13		0.50		99.74
Taxi	14		0.50		99.72
Climb	1		60.00		97.34
Climb	2		53.40		95.22
Climb	3		50.40		93.22
Cruise	1		15.00		92.63
Cruise	2		15.00		92.03
Cruise	3		15.00		91.44
Descend	1		0.10		91.

In [9]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1000  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_drop_rate = 2  # Minimum SoC drop rate in % per minute for climb

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 1, 10
power_min_climb, power_max_climb = 2, 70
power_min_cruise, power_max_cruise = 10, 50
power_min_descend, power_max_descend = 5, 25

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.277
duration_climb_min, duration_climb_max = 0.006, 0.047
duration_cruise_min, duration_cruise_max = 0.003, 0.503
duration_descend_min, duration_descend_max = 0.005, 0.052

# SoC drop limits for each stage
soc_max_drop_taxi = 3
soc_max_drop_climb = 9
soc_max_drop_cruise = 48
soc_max_drop_descend = 3

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"

# Altitude constraint: Ensure SoC drop during climb is enough to reach target altitude
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 * altitude_rate >= altitude_target, "Altitude_Climb_Target"

# Minimum SoC drop rate constraint for climb to achieve rate of altitude gain
for t in range(int(duration_climb_max / delta_t)):
    model += (P_climb[t] * delta_t / battery_capacity) * 100 >= min_soc_drop_rate, f"Min_SoC_Drop_Rate_Climb_{t}"

# Solve the problem
model.solve()

# Print results
# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")

Status: 1
Optimal Power Settings per Time Step:
Taxi Power: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] kW
Climb Power: [58.8, 25.2] kW
Cruise Power: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] kW
Descend Power: [5.0, 5.0, 5.0] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.26666667 hours ( 16.0 minutes)
Climb Duration: 0.033333333 hours ( 2.0 minutes)
Cruise Duration: 0.5 hours ( 30.0 minutes)
Descend Duration: 0.05 hours ( 3.0 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		1.00		99.92
Taxi	2		1.00		99.84
Taxi	3		1.00		99.76
Taxi	4		1.00		99.68
Taxi	5		1.00		99.60
Taxi	6		1.00		99.52
Taxi	7		1.00		99.44
Taxi	8		1.00		99.37
Taxi	9		1.00		99.29
Taxi	10		1.00		99.21
Taxi	11		1.00		99.13
Taxi	12		1.00		99.05
Taxi	13		1.00		98.97
Taxi	14		1.00		9

## Working model for circuit flights

In [10]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value, LpStatus

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_Circuits", LpMaximize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_for_circuit = 20  # Minimum SoC required to start a circuit in percentage
max_circuits = 2  # Maximum number of circuits allowed

# Phase-specific power bounds (in kW)
power_min = {"taxi": 0.5, "climb": 40, "cruise": 15, "descend": 5}
power_max = {"taxi": 2, "climb": 50, "cruise": 25, "descend": 10}

# Bounds for phase durations (in hours)
duration_bounds = {
    "taxi": (0.003, 0.2),
    "climb": (0.01, 0.05),
    "cruise": (0.01, 0.05),
    "descend": (0.005, 0.05)
}

# Decision variables
num_circuits = LpVariable("Number_of_Circuits", lowBound=0, upBound=max_circuits, cat="Integer")
soc_remaining = LpVariable.dicts("SoC_Remaining", range(max_circuits + 1), lowBound=0, upBound=100)
soc_drop_actual = LpVariable.dicts("SoC_Drop_Actual", range(1, max_circuits + 1), lowBound=0, upBound=100)
is_circuit_active = LpVariable.dicts("Is_Circuit_Active", range(1, max_circuits + 1), cat="Binary")

# Phase durations, power, and energy as decision variables for each circuit
phase_durations = {
    (circuit, phase): LpVariable(
        f"Duration_Circuit_{circuit}_{phase.capitalize()}",
        lowBound=duration_bounds[phase][0],
        upBound=duration_bounds[phase][1]
    )
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}
phase_power = {
    (circuit, phase): LpVariable(
        f"Power_Circuit_{circuit}_{phase.capitalize()}",
        lowBound=power_min[phase],
        upBound=power_max[phase]
    )
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}
energy_used_phase = {
    (circuit, phase): LpVariable(f"Energy_Used_Circuit_{circuit}_{phase.capitalize()}", lowBound=0)
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}

# SoC drop per phase and circuit
soc_drop_phase = {
    (circuit, phase): LpVariable(f"SoC_Drop_Circuit_{circuit}_{phase.capitalize()}", lowBound=0, upBound=100)
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}

# Objective: Maximize the number of circuits
model += lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Objective"

# Initial SoC constraint
model += soc_remaining[0] == initial_soc, "Initial_SoC"

# Phase-specific constraints
for circuit in range(1, max_circuits + 1):
    for phase in duration_bounds:
        duration_var = phase_durations[circuit, phase]
        power_var = phase_power[circuit, phase]
        energy_var = energy_used_phase[circuit, phase]

        # Energy = Power * Duration linearization
        model += energy_var == power_var * duration_var, f"Energy_Linear_C{circuit}_P{phase}"

        # Link energy to SoC drop (linearized)
        model += energy_var * 100 == soc_drop_phase[circuit, phase] * battery_capacity, f"SoC_Drop_Link_C{circuit}_P{phase}"

        # Ensure inactive circuits have zero duration, power, and energy
        model += duration_var <= duration_bounds[phase][1] * is_circuit_active[circuit], f"Duration_Active_C{circuit}_P{phase}"
        model += power_var <= power_max[phase] * is_circuit_active[circuit], f"Power_Active_C{circuit}_P{phase}"
        model += energy_var <= power_max[phase] * duration_bounds[phase][1] * is_circuit_active[circuit], f"Energy_Active_C{circuit}_P{phase}"

    # Total SoC drop for each circuit
    model += soc_drop_actual[circuit] == lpSum([soc_drop_phase[circuit, phase] for phase in duration_bounds]), f"SoC_Total_Circuit_{circuit}"

    # Update SoC after each circuit
    model += soc_remaining[circuit] == soc_remaining[circuit - 1] - soc_drop_actual[circuit], f"SoC_Update_Circuit_{circuit}"

    # Ensure minimum SoC threshold for active circuits
    model += soc_remaining[circuit] >= min_soc_for_circuit * is_circuit_active[circuit], f"Min_SoC_Circuit_{circuit}"

# Total number of circuits constraint
model += num_circuits == lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Total_Number_of_Circuits"

# Battery capacity constraint
model += lpSum([soc_drop_actual[c] for c in range(1, max_circuits + 1)]) <= initial_soc, "Battery_Capacity"

# Altitude constraint: Ensure climb phase achieves target altitude
model += lpSum([soc_drop_phase[c, "climb"] for c in range(1, max_circuits + 1)]) * altitude_rate >= altitude_target, "Altitude_Target"

# Solve the problem
model.solve()

# Display results
print("Status:", LpStatus[model.status])
print(f"Number of Circuits: {value(num_circuits)}")
print("\nPhase Details (Duration, Power, Energy, SoC Drop) for Each Circuit:")
for circuit in range(1, max_circuits + 1):
    print(f"Circuit {circuit}:")
    for phase in duration_bounds:
        duration_val = value(phase_durations[circuit, phase])
        power_val = value(phase_power[circuit, phase])
        energy_val = value(energy_used_phase[circuit, phase])
        soc_drop_val = value(soc_drop_phase[circuit, phase])
        if duration_val and power_val and energy_val and soc_drop_val is not None:
            print(f"  {phase.capitalize()}: Duration = {duration_val:.3f} hours, "
                  f"Power = {power_val:.2f} kW, "
                  f"Energy = {energy_val:.2f} kWh, "
                  f"SoC Drop = {soc_drop_val:.2f}%")


TypeError: Non-constant expressions cannot be multiplied

In [ ]:
print("\nConstraint Satisfaction Check:")
for name, constraint in model.constraints.items():
    slack = constraint.slack
    print(f"Constraint '{name}': Slack = {slack:.4f}")


# Circuit Flights

____

# 1. Using a determinitsic drop rate per circuit

In [11]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_Circuits", LpMaximize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_for_circuit = 20  # Minimum SoC required to start a circuit in percentage
max_circuits = 25  # Maximum number of circuits allowed

# Phase-specific parameters
drop_rate = {
    "taxi": 5,    # % drop per hour
    "climb": 50,  # % drop per hour
    "cruise": 20, # % drop per hour
    "descend": 5  # % drop per hour
}

# Bounds for phase durations (in hours)
duration_bounds = {
    "taxi": (0.003, 0.2),
    "climb": (0.01, 0.05),
    "cruise": (0.01, 0.1),
    "descend": (0.005, 0.05)
}

# Decision variables
num_circuits = LpVariable("Number_of_Circuits", lowBound=0, upBound=max_circuits, cat="Integer")
soc_remaining = LpVariable.dicts("SoC_Remaining", range(max_circuits + 1), lowBound=0, upBound=100)
soc_drop_actual = LpVariable.dicts("SoC_Drop_Actual", range(1, max_circuits + 1), lowBound=0, upBound=100)
is_circuit_active = LpVariable.dicts("Is_Circuit_Active", range(1, max_circuits + 1), cat="Binary")

# Phase durations as decision variables
phase_durations = {phase: LpVariable(f"Duration_{phase.capitalize()}", lowBound=duration_bounds[phase][0], upBound=duration_bounds[phase][1]) for phase in drop_rate}

# SoC drop per phase and circuit
soc_drop_phase = {
    (circuit, phase): LpVariable(f"SoC_Drop_Circuit_{circuit}_{phase.capitalize()}", lowBound=0, upBound=100)
    for circuit in range(1, max_circuits + 1)
    for phase in drop_rate
}

# Objective: Maximize the number of circuits
model += lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Objective"

# Initial SoC constraint
model += soc_remaining[0] == initial_soc, "Initial_SoC"

# Phase-specific SoC drops
for circuit in range(1, max_circuits + 1):
    for phase in drop_rate:
        duration_var = phase_durations[phase]
        drop_rate_var = drop_rate[phase]
        max_soc_drop_phase = drop_rate_var * duration_bounds[phase][1]
        # Linearize the relationship
        model += soc_drop_phase[circuit, phase] <= drop_rate_var * duration_var, f"SoC_Phase_Max_{circuit}_{phase.capitalize()}"
        model += soc_drop_phase[circuit, phase] <= max_soc_drop_phase * is_circuit_active[circuit], f"SoC_Phase_Active_Upper_{circuit}_{phase.capitalize()}"
        model += soc_drop_phase[circuit, phase] >= drop_rate_var * duration_var - max_soc_drop_phase * (1 - is_circuit_active[circuit]), f"SoC_Phase_Active_Lower_{circuit}_{phase.capitalize()}"

    # Total SoC drop for each circuit
    model += soc_drop_actual[circuit] == lpSum([soc_drop_phase[circuit, phase] for phase in drop_rate]), f"SoC_Total_Circuit_{circuit}"

    # Update SoC after each circuit
    model += soc_remaining[circuit] == soc_remaining[circuit - 1] - soc_drop_actual[circuit], f"SoC_Update_Circuit_{circuit}"

    # Ensure minimum SoC threshold for active circuits
    model += soc_remaining[circuit] >= min_soc_for_circuit * is_circuit_active[circuit], f"Min_SoC_Circuit_{circuit}"

# Total number of circuits constraint
model += num_circuits == lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Total_Number_of_Circuits"

# Battery capacity constraint
model += lpSum([soc_drop_actual[c] for c in range(1, max_circuits + 1)]) <= initial_soc, "Battery_Capacity"

# Altitude constraint: Ensure climb phase achieves target altitude
model += lpSum([soc_drop_phase[c, "climb"] for c in range(1, max_circuits + 1)]) * altitude_rate >= altitude_target, "Altitude_Target"

# Solve the problem
model.solve()

# Print results
print("Status:", model.status)
# Display decision variables
print("Decision Variables:")
print(f"Number of Circuits: {value(num_circuits)}")
print(f"Phase Durations (hours):")
for phase in phase_durations:
    print(f"  {phase.capitalize()}: {value(phase_durations[phase]):.3f}")

# Display SoC values
print("\nState of Charge After Each Circuit:")
for circuit in range(max_circuits + 1):
    if circuit <= value(num_circuits):
        print(f"Circuit {circuit}: {value(soc_remaining[circuit]):.2f}% SoC")
    else:
        print(f"Circuit {circuit}: Not Performed")

# Display total energy used
total_energy_used = sum(value(soc_drop_actual[c]) for c in range(1, max_circuits + 1))
print(f"\nTotal Energy Used: {total_energy_used:.2f}% of battery capacity")


Status: 1
Decision Variables:
Number of Circuits: 25.0
Phase Durations (hours):
  Taxi: 0.200
  Climb: 0.010
  Cruise: 0.084
  Descend: 0.005

State of Charge After Each Circuit:
Circuit 0: 100.00% SoC
Circuit 1: 96.80% SoC
Circuit 2: 93.60% SoC
Circuit 3: 90.40% SoC
Circuit 4: 87.20% SoC
Circuit 5: 84.00% SoC
Circuit 6: 80.80% SoC
Circuit 7: 77.60% SoC
Circuit 8: 74.40% SoC
Circuit 9: 71.20% SoC
Circuit 10: 68.00% SoC
Circuit 11: 64.80% SoC
Circuit 12: 61.60% SoC
Circuit 13: 58.40% SoC
Circuit 14: 55.20% SoC
Circuit 15: 52.00% SoC
Circuit 16: 48.80% SoC
Circuit 17: 45.60% SoC
Circuit 18: 42.40% SoC
Circuit 19: 39.20% SoC
Circuit 20: 36.00% SoC
Circuit 21: 32.80% SoC
Circuit 22: 29.60% SoC
Circuit 23: 26.40% SoC
Circuit 24: 23.20% SoC
Circuit 25: 20.00% SoC

Total Energy Used: 80.00% of battery capacity


In [12]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_Circuits", LpMaximize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_for_circuit = 20  # Minimum SoC required to start a circuit in percentage
max_circuits = 31  # Maximum number of circuits allowed
drop_rate_decay = 0.9  # Decay factor for drop rate per circuit

# Initial drop rates per phase
initial_drop_rate = {
    "taxi": 5,    # % drop per hour
    "climb": 50,  # % drop per hour
    "cruise": 20, # % drop per hour
    "descend": 5  # % drop per hour
}

# Bounds for phase durations (in hours)
duration_bounds = {
    "taxi": (0.003, 0.2),
    "climb": (0.01, 0.05),
    "cruise": (0.01, 0.1),
    "descend": (0.005, 0.05)
}

# Decision variables
num_circuits = LpVariable("Number_of_Circuits", lowBound=0, upBound=max_circuits, cat="Integer")
soc_remaining = LpVariable.dicts("SoC_Remaining", range(max_circuits + 1), lowBound=0, upBound=100)
soc_drop_actual = LpVariable.dicts("SoC_Drop_Actual", range(1, max_circuits + 1), lowBound=0, upBound=100)
is_circuit_active = LpVariable.dicts("Is_Circuit_Active", range(1, max_circuits + 1), cat="Binary")

# Phase durations as decision variables
phase_durations = {phase: LpVariable(f"Duration_{phase.capitalize()}", lowBound=duration_bounds[phase][0], upBound=duration_bounds[phase][1]) for phase in initial_drop_rate}

# Dynamic drop rates for each circuit and phase
adjusted_drop_rate = {
    (circuit, phase): LpVariable(f"Drop_Rate_Circuit_{circuit}_{phase.capitalize()}", lowBound=0)
    for circuit in range(1, max_circuits + 1)
    for phase in initial_drop_rate
}

# Auxiliary variables for the product of drop rate and duration
soc_drop_product = {
    (circuit, phase): LpVariable(f"SoC_Drop_Product_Circuit_{circuit}_{phase.capitalize()}", lowBound=0, upBound=100)
    for circuit in range(1, max_circuits + 1)
    for phase in initial_drop_rate
}

# Phase-specific SoC drops
soc_drop_phase = {
    (circuit, phase): LpVariable(f"SoC_Drop_Circuit_{circuit}_{phase.capitalize()}", lowBound=0, upBound=100)
    for circuit in range(1, max_circuits + 1)
    for phase in initial_drop_rate
}

# Objective: Maximize the number of circuits
model += lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Objective"

# Initial SoC constraint
model += soc_remaining[0] == initial_soc, "Initial_SoC"

# Define adjusted drop rates and apply constraints
for circuit in range(1, max_circuits + 1):
    for phase in initial_drop_rate:
        # Adjust the drop rate for the circuit
        model += adjusted_drop_rate[circuit, phase] == initial_drop_rate[phase] * (drop_rate_decay ** (circuit - 1)), f"Adjusted_Drop_Rate_{circuit}_{phase.capitalize()}"

        # Define the product of adjusted drop rate and duration
        model += soc_drop_product[circuit, phase] <= adjusted_drop_rate[circuit, phase] * duration_bounds[phase][1], f"Soc_Drop_Product_Upper_{circuit}_{phase.capitalize()}"
        model += soc_drop_product[circuit, phase] <= phase_durations[phase] * initial_drop_rate[phase], f"Soc_Drop_Product_Duration_{circuit}_{phase.capitalize()}"

        # Constrain SoC drop phase to the auxiliary variable
        model += soc_drop_phase[circuit, phase] == soc_drop_product[circuit, phase], f"SoC_Drop_Phase_{circuit}_{phase.capitalize()}"

    # Total SoC drop for each circuit
    model += soc_drop_actual[circuit] == lpSum([soc_drop_phase[circuit, phase] for phase in initial_drop_rate]), f"SoC_Total_Circuit_{circuit}"

    # Update SoC after each circuit
    model += soc_remaining[circuit] == soc_remaining[circuit - 1] - soc_drop_actual[circuit], f"SoC_Update_Circuit_{circuit}"

    # Ensure minimum SoC threshold for active circuits
    model += soc_remaining[circuit] >= min_soc_for_circuit * is_circuit_active[circuit], f"Min_SoC_Circuit_{circuit}"

# Total number of circuits constraint
model += num_circuits == lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Total_Number_of_Circuits"

# Battery capacity constraint
model += lpSum([soc_drop_actual[c] for c in range(1, max_circuits + 1)]) <= initial_soc, "Battery_Capacity"

# Altitude constraint: Ensure climb phase achieves target altitude
model += lpSum([soc_drop_phase[c, "climb"] for c in range(1, max_circuits + 1)]) * altitude_rate >= altitude_target, "Altitude_Target"

# Solve the problem
model.solve()

# Print results
print("Status:", model.status)
# Display decision variables
print("Decision Variables:")
print(f"Number of Circuits: {value(num_circuits)}")
print(f"Phase Durations (hours):")
for phase in phase_durations:
    print(f"  {phase.capitalize()}: {value(phase_durations[phase]):.3f}")

# Display SoC values
print("\nState of Charge After Each Circuit:")
for circuit in range(max_circuits + 1):
    if circuit <= value(num_circuits):
        print(f"Circuit {circuit}: {value(soc_remaining[circuit]):.2f}% SoC")
    else:
        print(f"Circuit {circuit}: Not Performed")

# Display total energy used
total_energy_used = sum(value(soc_drop_actual[c]) for c in range(1, max_circuits + 1))
print(f"\nTotal Energy Used: {total_energy_used:.2f}% of battery capacity")


Status: 1
Decision Variables:
Number of Circuits: 31.0
Phase Durations (hours):
  Taxi: 0.200
  Climb: 0.050
  Cruise: 0.100
  Descend: 0.050

State of Charge After Each Circuit:
Circuit 0: 100.00% SoC
Circuit 1: 94.25% SoC
Circuit 2: 89.08% SoC
Circuit 3: 84.42% SoC
Circuit 4: 80.23% SoC
Circuit 5: 76.45% SoC
Circuit 6: 73.06% SoC
Circuit 7: 70.00% SoC
Circuit 8: 67.25% SoC
Circuit 9: 64.78% SoC
Circuit 10: 62.55% SoC
Circuit 11: 60.54% SoC
Circuit 12: 58.74% SoC
Circuit 13: 57.12% SoC
Circuit 14: 55.65% SoC
Circuit 15: 54.34% SoC
Circuit 16: 53.15% SoC
Circuit 17: 52.09% SoC
Circuit 18: 51.13% SoC
Circuit 19: 50.27% SoC
Circuit 20: 49.49% SoC
Circuit 21: 48.79% SoC
Circuit 22: 48.16% SoC
Circuit 23: 47.60% SoC
Circuit 24: 47.09% SoC
Circuit 25: 46.63% SoC
Circuit 26: 46.39% SoC
Circuit 27: 46.18% SoC
Circuit 28: 46.00% SoC
Circuit 29: 45.83% SoC
Circuit 30: 45.83% SoC
Circuit 31: 45.83% SoC

Total Energy Used: 54.17% of battery capacity


### Plot the chart of how the time varies per phase as the number of circuits changes

### 2.

In [13]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_Circuits", LpMaximize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_for_circuit = 20  # Minimum SoC required to start a circuit in percentage
max_circuits = 15  # Maximum number of circuits allowed
BigM = 1000  # Large constant for linearization

# Phase-specific power bounds (in kW)
power_min = {"taxi": 0.5, "climb": 40, "cruise": 15, "descend": 5}
power_max = {"taxi": 2, "climb": 50, "cruise": 25, "descend": 10}

# Bounds for phase durations (in hours)
duration_bounds = {
    "taxi": (0.003, 0.2),
    "climb": (0.01, 0.05),
    "cruise": (0.01, 0.2),
    "descend": (0.005, 0.05)
}

# Decision variables
num_circuits = LpVariable("Number_of_Circuits", lowBound=0, upBound=max_circuits, cat="Integer")
soc_remaining = LpVariable.dicts("SoC_Remaining", range(max_circuits + 1), lowBound=0, upBound=100)
soc_drop_actual = LpVariable.dicts("SoC_Drop_Actual", range(1, max_circuits + 1), lowBound=0, upBound=100)
is_circuit_active = LpVariable.dicts("Is_Circuit_Active", range(1, max_circuits + 1), cat="Binary")

# Phase durations, power, and energy as decision variables for each circuit
phase_durations = {
    (circuit, phase): LpVariable(
        f"Duration_Circuit_{circuit}_{phase.capitalize()}",
        lowBound=duration_bounds[phase][0],
        upBound=duration_bounds[phase][1]
    )
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}
phase_power = {
    (circuit, phase): LpVariable(
        f"Power_Circuit_{circuit}_{phase.capitalize()}",
        lowBound=power_min[phase],
        upBound=power_max[phase]
    )
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}
energy_used_phase = {
    (circuit, phase): LpVariable(f"Energy_Used_Circuit_{circuit}_{phase.capitalize()}", lowBound=0)
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}

# SoC drop per phase and circuit
soc_drop_phase = {
    (circuit, phase): LpVariable(f"SoC_Drop_Circuit_{circuit}_{phase.capitalize()}", lowBound=0, upBound=100)
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}

# Objective: Maximize the number of circuits
model += lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Objective"

# Initial SoC constraint
model += soc_remaining[0] == initial_soc, "Initial_SoC"

# Phase-specific constraints
for circuit in range(1, max_circuits + 1):
    for phase in duration_bounds:
        duration_var = phase_durations[circuit, phase]
        power_var = phase_power[circuit, phase]
        energy_var = energy_used_phase[circuit, phase]

        # Linearize energy = power * duration
        model += energy_var <= power_var * duration_bounds[phase][1], f"Energy_Upper_Power_C{circuit}_P{phase}"
        model += energy_var <= duration_var * power_max[phase], f"Energy_Upper_Duration_C{circuit}_P{phase}"
        model += energy_var >= 0, f"Energy_Lower_Zero_C{circuit}_P{phase}"
        model += energy_var <= power_max[phase] * duration_bounds[phase][1] * is_circuit_active[circuit], f"Energy_Active_C{circuit}_P{phase}"

        # Link energy to SoC drop
        model += soc_drop_phase[circuit, phase] * battery_capacity == energy_var * 100, f"SoC_Drop_Link_C{circuit}_P{phase}"

        # Ensure inactive circuits have zero duration, power, and energy
        model += duration_var <= duration_bounds[phase][1] * is_circuit_active[circuit], f"Duration_Active_C{circuit}_P{phase}"
        model += power_var <= power_max[phase] * is_circuit_active[circuit], f"Power_Active_C{circuit}_P{phase}"
        model += soc_drop_phase[circuit, phase] <= BigM * is_circuit_active[circuit], f"SoC_Phase_Active_C{circuit}_P{phase}"

    # Total SoC drop for each circuit
    model += soc_drop_actual[circuit] == lpSum([soc_drop_phase[circuit, phase] for phase in duration_bounds]), f"SoC_Total_Circuit_{circuit}"

    # Update SoC after each circuit
    model += soc_remaining[circuit] == soc_remaining[circuit - 1] - soc_drop_actual[circuit], f"SoC_Update_Circuit_{circuit}"

    # Ensure minimum SoC threshold for active circuits
    model += soc_remaining[circuit] >= min_soc_for_circuit * is_circuit_active[circuit], f"Min_SoC_Circuit_{circuit}"

# Total number of circuits constraint
model += num_circuits == lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Total_Number_of_Circuits"

# Battery capacity constraint
model += lpSum([soc_drop_actual[c] for c in range(1, max_circuits + 1)]) <= initial_soc, "Battery_Capacity"

# Altitude constraint: Ensure climb phase achieves target altitude
model += lpSum([soc_drop_phase[c, "climb"] for c in range(1, max_circuits + 1)]) * altitude_rate >= altitude_target, "Altitude_Target"

# Solve the problem
model.solve()

# Print results
print("Status:", model.status)
print(f"Number of Circuits: {value(num_circuits)}")

# Display phase durations, power, and energy for each circuit
print("\nPhase Details (Duration, Power, Energy, SoC Drop) for Each Circuit:")
for circuit in range(1, max_circuits + 1):
    print(f"Circuit {circuit}:")
    for phase in duration_bounds:
        print(f"  {phase.capitalize()}: Duration = {value(phase_durations[circuit, phase]):.3f} hours, "
              f"Power = {value(phase_power[circuit, phase]):.2f} kW, "
              f"Energy = {value(energy_used_phase[circuit, phase]):.2f} kWh, "
              f"SoC Drop = {value(soc_drop_phase[circuit, phase]):.2f}%")

# Display start and end SoC for each circuit
print("\nState of Charge for Each Circuit:")
for circuit in range(1, max_circuits + 1):
    start_soc = value(soc_remaining[circuit - 1]) if circuit > 1 else initial_soc
    end_soc = value(soc_remaining[circuit])
    if start_soc is not None and end_soc is not None and end_soc >= min_soc_for_circuit:
        print(f"Circuit {circuit}: Start SoC = {start_soc:.2f}%, End SoC = {end_soc:.2f}%")
    else:
        print(f"Circuit {circuit}: Not Performed")

# Display total energy used
total_energy_used = sum(value(soc_drop_actual[c]) for c in range(1, max_circuits + 1))
print(f"\nTotal Energy Used: {total_energy_used:.2f}% of battery capacity")


Status: 1
Number of Circuits: 15.0

Phase Details (Duration, Power, Energy, SoC Drop) for Each Circuit:
Circuit 1:
  Taxi: Duration = 0.200 hours, Power = 2.00 kW, Energy = 0.40 kWh, SoC Drop = 1.90%
  Climb: Duration = 0.050 hours, Power = 50.00 kW, Energy = 2.50 kWh, SoC Drop = 11.90%
  Cruise: Duration = 0.200 hours, Power = 25.00 kW, Energy = 5.00 kWh, SoC Drop = 23.81%
  Descend: Duration = 0.050 hours, Power = 10.00 kW, Energy = 0.50 kWh, SoC Drop = 2.38%
Circuit 2:
  Taxi: Duration = 0.200 hours, Power = 2.00 kW, Energy = 0.40 kWh, SoC Drop = 1.90%
  Climb: Duration = 0.050 hours, Power = 50.00 kW, Energy = 2.50 kWh, SoC Drop = 11.90%
  Cruise: Duration = 0.200 hours, Power = 25.00 kW, Energy = 5.00 kWh, SoC Drop = 23.81%
  Descend: Duration = 0.050 hours, Power = 10.00 kW, Energy = 0.50 kWh, SoC Drop = 2.38%
Circuit 3:
  Taxi: Duration = 0.200 hours, Power = 2.00 kW, Energy = 0.00 kWh, SoC Drop = 0.00%
  Climb: Duration = 0.050 hours, Power = 50.00 kW, Energy = 0.00 kWh, SoC Dr

In [14]:
# Feet gained per min based on the power setting used

In [15]:
print("\nConstraint Satisfaction Check:")
for name, constraint in model.constraints.items():
    slack = constraint.slack
    print(f"Constraint '{name}': Slack = {slack:.4f}")


Constraint Satisfaction Check:
Constraint 'Initial_SoC': Slack = -0.0000
Constraint 'Energy_Upper_Power_C1_Ptaxi': Slack = 0.0000
Constraint 'Energy_Upper_Duration_C1_Ptaxi': Slack = 0.0000
Constraint 'Energy_Lower_Zero_C1_Ptaxi': Slack = -0.4000
Constraint 'Energy_Active_C1_Ptaxi': Slack = 0.0000
Constraint 'SoC_Drop_Link_C1_Ptaxi': Slack = -0.0000
Constraint 'Duration_Active_C1_Ptaxi': Slack = -0.0000
Constraint 'Power_Active_C1_Ptaxi': Slack = -0.0000
Constraint 'SoC_Phase_Active_C1_Ptaxi': Slack = 998.0952
Constraint 'Energy_Upper_Power_C1_Pclimb': Slack = -0.0000
Constraint 'Energy_Upper_Duration_C1_Pclimb': Slack = -0.0000
Constraint 'Energy_Lower_Zero_C1_Pclimb': Slack = -2.5000
Constraint 'Energy_Active_C1_Pclimb': Slack = -0.0000
Constraint 'SoC_Drop_Link_C1_Pclimb': Slack = -0.0000
Constraint 'Duration_Active_C1_Pclimb': Slack = -0.0000
Constraint 'Power_Active_C1_Pclimb': Slack = -0.0000
Constraint 'SoC_Phase_Active_C1_Pclimb': Slack = 988.0952
Constraint 'Energy_Upper_Powe

# With power and time

In [16]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value, LpStatus

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_Circuits", LpMaximize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_for_circuit = 20  # Minimum SoC required to start a circuit in percentage
max_circuits = 2  # Maximum number of circuits allowed

# Phase-specific power bounds (in kW)
power_min = {"taxi": 0.5, "climb": 40, "cruise": 15, "descend": 5}
power_max = {"taxi": 2, "climb": 50, "cruise": 25, "descend": 10}

# Bounds for phase durations (in hours)
duration_bounds = {
    "taxi": (0.003, 0.2),
    "climb": (0.01, 0.05),
    "cruise": (0.01, 0.05),
    "descend": (0.005, 0.05)
}

# Decision variables
num_circuits = LpVariable("Number_of_Circuits", lowBound=0, upBound=max_circuits, cat="Integer")
soc_remaining = LpVariable.dicts("SoC_Remaining", range(max_circuits + 1), lowBound=0, upBound=100)
soc_drop_actual = LpVariable.dicts("SoC_Drop_Actual", range(1, max_circuits + 1), lowBound=0, upBound=100)
is_circuit_active = LpVariable.dicts("Is_Circuit_Active", range(1, max_circuits + 1), cat="Binary")

# Phase durations, power, and energy as decision variables for each circuit
phase_durations = {
    (circuit, phase): LpVariable(
        f"Duration_Circuit_{circuit}_{phase.capitalize()}",
        lowBound=duration_bounds[phase][0],
        upBound=duration_bounds[phase][1]
    )
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}
phase_power = {
    (circuit, phase): LpVariable(
        f"Power_Circuit_{circuit}_{phase.capitalize()}",
        lowBound=power_min[phase],
        upBound=power_max[phase]
    )
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}
energy_used_phase = {
    (circuit, phase): LpVariable(f"Energy_Used_Circuit_{circuit}_{phase.capitalize()}", lowBound=0)
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}

# SoC drop per phase and circuit
soc_drop_phase = {
    (circuit, phase): LpVariable(f"SoC_Drop_Circuit_{circuit}_{phase.capitalize()}", lowBound=0, upBound=100)
    for circuit in range(1, max_circuits + 1)
    for phase in duration_bounds
}

# Objective: Maximize the number of circuits
model += lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Objective"

# Initial SoC constraint
model += soc_remaining[0] == initial_soc, "Initial_SoC"

# Phase-specific constraints
for circuit in range(1, max_circuits + 1):
    for phase in duration_bounds:
        duration_var = phase_durations[circuit, phase]
        power_var = phase_power[circuit, phase]
        energy_var = energy_used_phase[circuit, phase]

        # Energy = Power * Duration linearization
        model += energy_var == power_var * duration_var, f"Energy_Linear_C{circuit}_P{phase}"

        # Link energy to SoC drop (linearized)
        model += energy_var * 100 == soc_drop_phase[circuit, phase] * battery_capacity, f"SoC_Drop_Link_C{circuit}_P{phase}"

        # Ensure inactive circuits have zero duration, power, and energy
        model += duration_var <= duration_bounds[phase][1] * is_circuit_active[circuit], f"Duration_Active_C{circuit}_P{phase}"
        model += power_var <= power_max[phase] * is_circuit_active[circuit], f"Power_Active_C{circuit}_P{phase}"
        model += energy_var <= power_max[phase] * duration_bounds[phase][1] * is_circuit_active[circuit], f"Energy_Active_C{circuit}_P{phase}"

    # Total SoC drop for each circuit
    model += soc_drop_actual[circuit] == lpSum([soc_drop_phase[circuit, phase] for phase in duration_bounds]), f"SoC_Total_Circuit_{circuit}"

    # Update SoC after each circuit
    model += soc_remaining[circuit] == soc_remaining[circuit - 1] - soc_drop_actual[circuit], f"SoC_Update_Circuit_{circuit}"

    # Ensure minimum SoC threshold for active circuits
    model += soc_remaining[circuit] >= min_soc_for_circuit * is_circuit_active[circuit], f"Min_SoC_Circuit_{circuit}"

# Total number of circuits constraint
model += num_circuits == lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Total_Number_of_Circuits"

# Battery capacity constraint
model += lpSum([soc_drop_actual[c] for c in range(1, max_circuits + 1)]) <= initial_soc, "Battery_Capacity"

# Altitude constraint: Ensure climb phase achieves target altitu


TypeError: Non-constant expressions cannot be multiplied

In [17]:
print("\nConstraint Satisfaction Check:")
for name, constraint in model.constraints.items():
    slack = constraint.slack
    print(f"Constraint '{name}': Slack = {slack:.4f}")



Constraint Satisfaction Check:


TypeError: unsupported format string passed to NoneType.__format__